## BLE-RSSI Dataset for Indoor localization using MinIO/ Workspace-volume storage 

The dataset was created using the RSSI readings of an array of 13 ibeacons in the first floor of Waldo Library, Western Michigan University. Data was collected using iPhone 6S. The dataset contains two sub-datasets: a labeled dataset (1420 instances) and an unlabeled dataset (5191 instances). The recording was performed during the operational hours of the library. For the labeled dataset, the input data contains the location (label column), a timestamp, followed by RSSI readings of 13 iBeacons. RSSI measurements are negative values. Bigger RSSI values indicate closer proximity to a given iBeacon (e.g., RSSI of -65 represent a closer distance to a given iBeacon compared to RSSI of -85). For out-of-range iBeacons, the RSSI is indicated by -200. The locations related to RSSI readings are combined in one column consisting a letter for the column and a number for the row of the position.

## Clone Cisco Kubeflow Starter pack repository

In [ ]:
BRANCH_NAME="dev" #Provide git branch name "master" or "dev"
! git clone -b $BRANCH_NAME https://github.com/CiscoAI/cisco-kubeflow-starter-pack.git

## Create requirements.txt

In [1]:
%%writefile requirements.txt
pandas
numpy
seldon-core
tornado>=6.0.3
kubeflow-tfjob
kubeflow-fairing
tensorflow==1.14.0
kubernetes==10.0.1
minio

Overwriting requirements.txt


## Install packages

In [2]:
!pip install -r requirements.txt --user

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


## Restart Notebook kernel

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

## Set the name of the model store

In [28]:
model_store = ''

In [29]:
if not model_store or model_store not in ('minio','workspace-vol'):
     raise ValueError("Set the name of the model store to be used: minio/workspace-vol")

## Import libraries

In [30]:
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil
import os
import logging
import re
import yaml
from minio import Minio
from kubernetes import client as k8s_client
from kubernetes.client import rest as k8s_rest
from kubernetes import config as k8s_config
from kubernetes.client.rest import ApiException
from kubeflow.fairing.cloud.k8s import MinioUploader
from kubeflow.fairing.builders.cluster.minio_context import MinioContextSource

## Prerequisites

### Create minIO secret & serviceaccount

In [31]:
if model_store == 'minio':
    
    # Create MinIO secret
    minio_secret = f"""apiVersion: v1
kind: Secret
metadata:
  name: miniosecret
  annotations:
     serving.kubeflow.org/s3-endpoint: minio-service.kubeflow:9000 # replace with your s3 endpoint
     serving.kubeflow.org/s3-usehttps: "0" # by default 1, for testing with minio you need to set to 0
type: Opaque
stringData:
  awsAccessKeyID: minio
  awsSecretAccessKey: minio123
"""

    minio_secret = yaml.safe_load(minio_secret)
    with open('minio-secret.yaml', 'w') as file:
        yaml_doc_secret = yaml.dump(minio_secret,file)
        
    print("Creating Minio secret..... ")
    
    !kubectl apply -f minio-secret.yaml -n anonymous
    
    !kubectl get secrets -n anonymous | grep miniosecret
    
    #Create MinIO service account
    minio_serviceaccount = f"""apiVersion: v1
kind: ServiceAccount
metadata:
  name: minio-sa
secrets:
- name: miniosecret
"""
    
    minio_serviceaccount = yaml.safe_load(minio_serviceaccount)
    with open('minio-serviceaccount.yaml', 'w') as file:
        yaml_doc_sa = yaml.dump(minio_serviceaccount,file)

    print("Creating Minio service account.....")
    
    !kubectl apply -f minio-serviceaccount.yaml -n anonymous
    
    !kubectl get serviceaccount -n anonymous | grep minio-sa

Creating Minio secret..... 
secret/miniosecret created
miniosecret                  Opaque                                2      0s
Creating Minio service account.....
serviceaccount/minio-sa created
minio-sa         2         0s


## Connect to minIO service & create MinIO bucket

In [32]:
if model_store == 'minio':
        
        #Connect to minIO service using credentials
        k8s_config.load_incluster_config()
        api_client = k8s_client.CoreV1Api()
        minio_service_endpoint = None

        try:
            minio_service_endpoint = api_client.read_namespaced_service(name='minio-service', namespace='kubeflow').spec.cluster_ip
        except ApiException as e:
            if e.status == 403:
                logging.warning(f"The service account doesn't have sufficient privileges "
                              f"to get the kubeflow minio-service. "
                              f"You will have to manually enter the minio cluster-ip. "
                              f"To make this function work ask someone with cluster "
                              f"priveleges to create an appropriate "
                              f"clusterrolebinding by running a command.\n"
                              f"kubectl create --namespace=kubeflow rolebinding "
                               "--clusterrole=kubeflow-view "
                               "--serviceaccount=${NAMESPACE}:default-editor "
                               "${NAMESPACE}-minio-view")
                logging.error("API access denied with reason: {e.reason}")

        s3_endpoint = minio_service_endpoint
        s3_endPoint = s3_endpoint+":9000"
        minio_endpoint = "http://"+s3_endPoint
        minio_username = "minio"
        minio_key = "minio123"
        minio_region = "us-east-1"
        print("Minio Endpoint returned:", minio_endpoint)
        
        # Define MinIO uploader
        minio_uploader = MinioUploader(endpoint_url=minio_endpoint, minio_secret=minio_username, minio_secret_key=minio_key, region_name=minio_region)
        
        # Create MinIO bucket
        minio_bucket = "minioblerssi"
        minio_uploader.create_bucket(minio_bucket)
        print("Minio bucket of %s created successfully or already exists" %minio_bucket)        

Minio Endpoint returned: http://10.106.203.39:9000
Minio bucket of minioblerssi created successfully or already exists


## Check GPUs availability

In [33]:
gpus = len(tf.config.experimental.list_physical_devices('GPU'))
if gpus == 0:
    print("Model will be trained using CPU")
elif gpus >= 0:
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    tf.config.experimental.list_physical_devices('GPU')
    print("Model will be trained using GPU")

Model will be trained using CPU


## Declare variables for model training

In [34]:
path = 'cisco-kubeflow-starter-pack/apps/networking/ble-localization/onprem/data/'
BLE_RSSI = pd.read_csv(os.path.join(path, 'iBeacon_RSSI_Labeled.csv')) #Labeled dataset
                       
# Configure model options
TF_DATA_DIR = os.getenv("TF_DATA_DIR", "/tmp/data/")
TF_MODEL_DIR = os.getenv("TF_MODEL_DIR", "blerssi/")
TF_EXPORT_DIR = os.getenv("TF_EXPORT_DIR", "blerssi/")
TF_MODEL_TYPE = os.getenv("TF_MODEL_TYPE", "DNN")
TF_TRAIN_STEPS = int(os.getenv("TF_TRAIN_STEPS", 5000))
TF_BATCH_SIZE = int(os.getenv("TF_BATCH_SIZE", 50))
TF_LEARNING_RATE = float(os.getenv("TF_LEARNING_RATE", 0.00001))


# Feature columns
COLUMNS = list(BLE_RSSI.columns)
FEATURES = COLUMNS[2:]
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]
  return input_columns

## BLERSSI Input Dataset

### Attribute Information

    location: The location of receiving RSSIs from ibeacons b3001 to b3013; 
              symbolic values showing the column and row of the location on the map (e.g., A01 stands for column A, row 1).
    date: Datetime in the format of ‘d-m-yyyy hh:mm:ss’
    b3001 - b3013: RSSI readings corresponding to the iBeacons; numeric, integers only.

In [35]:
BLE_RSSI.head(10)

,location,date,b3001,b3002,b3003,b3004,b3005,b3006,b3007,b3008,b3009,b3010,b3011,b3012,b3013
0,O02,10-18-2016 11:15:21,-200,-200,-200,-200,-200,-78,-200,-200,-200,-200,-200,-200,-200
1,P01,10-18-2016 11:15:19,-200,-200,-200,-200,-200,-78,-200,-200,-200,-200,-200,-200,-200
2,P01,10-18-2016 11:15:17,-200,-200,-200,-200,-200,-77,-200,-200,-200,-200,-200,-200,-200
3,P01,10-18-2016 11:15:15,-200,-200,-200,-200,-200,-77,-200,-200,-200,-200,-200,-200,-200
4,P01,10-18-2016 11:15:13,-200,-200,-200,-200,-200,-77,-200,-200,-200,-200,-200,-200,-200
5,P01,10-18-2016 11:15:11,-200,-200,-82,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
6,P01,10-18-2016 11:15:09,-200,-200,-80,-200,-200,-77,-200,-200,-200,-200,-200,-200,-200
7,P02,10-18-2016 11:15:07,-200,-200,-86,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
8,R01,10-18-2016 11:15:05,-200,-200,-200,-75,-200,-200,-200,-200,-200,-200,-200,-200,-200
9,R01,10-18-2016 11:15:03,-200,-200,-200,-75,-200,-200,-200,-200,-200,-200,-200,-200,-200


## Define serving input receiver function

In [36]:
feature_columns =  make_feature_cols()
inputs = {}
for feat in feature_columns:
  inputs[feat.name] = tf.placeholder(shape=[None], dtype=feat.dtype)
serving_input_receiver_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(inputs)

## Train and save BLE-RSSI Model

In [37]:
# Feature columns
COLUMNS = list(BLE_RSSI.columns)
FEATURES = COLUMNS[2:]
LABEL = [COLUMNS[0]]

b3001 = tf.feature_column.numeric_column(key='b3001',dtype=tf.float64)
b3002 = tf.feature_column.numeric_column(key='b3002',dtype=tf.float64)
b3003 = tf.feature_column.numeric_column(key='b3003',dtype=tf.float64)
b3004 = tf.feature_column.numeric_column(key='b3004',dtype=tf.float64)
b3005 = tf.feature_column.numeric_column(key='b3005',dtype=tf.float64)
b3006 = tf.feature_column.numeric_column(key='b3006',dtype=tf.float64)
b3007 = tf.feature_column.numeric_column(key='b3007',dtype=tf.float64)
b3008 = tf.feature_column.numeric_column(key='b3008',dtype=tf.float64)
b3009 = tf.feature_column.numeric_column(key='b3009',dtype=tf.float64)
b3010 = tf.feature_column.numeric_column(key='b3010',dtype=tf.float64)
b3011 = tf.feature_column.numeric_column(key='b3011',dtype=tf.float64)
b3012 = tf.feature_column.numeric_column(key='b3012',dtype=tf.float64)
b3013 = tf.feature_column.numeric_column(key='b3013',dtype=tf.float64)
feature_columns = [b3001, b3002, b3003, b3004, b3005, b3006, b3007, b3008, b3009, b3010, b3011, b3012, b3013]

df_full = pd.read_csv(os.path.join(path, "iBeacon_RSSI_Labeled.csv")) #Labeled dataset

# Input Data Preprocessing 
df_full = df_full.drop(['date'],axis = 1)
df_full[FEATURES] = (df_full[FEATURES])/(-200)


#Output Data Preprocessing
dict = {'O02': 0,'P01': 1,'P02': 2,'R01': 3,'R02': 4,'S01': 5,'S02': 6,'T01': 7,'U02': 8,'U01': 9,'J03': 10,'K03': 11,'L03': 12,'M03': 13,'N03': 14,'O03': 15,'P03': 16,'Q03': 17,'R03': 18,'S03': 19,'T03': 20,'U03': 21,'U04': 22,'T04': 23,'S04': 24,'R04': 25,'Q04': 26,'P04': 27,'O04': 28,'N04': 29,'M04': 30,'L04': 31,'K04': 32,'J04': 33,'I04': 34,'I05': 35,'J05': 36,'K05': 37,'L05': 38,'M05': 39,'N05': 40,'O05': 41,'P05': 42,'Q05': 43,'R05': 44,'S05': 45,'T05': 46,'U05': 47,'S06': 48,'R06': 49,'Q06': 50,'P06': 51,'O06': 52,'N06': 53,'M06': 54,'L06': 55,'K06': 56,'J06': 57,'I06': 58,'F08': 59,'J02': 60,'J07': 61,'I07': 62,'I10': 63,'J10': 64,'D15': 65,'E15': 66,'G15': 67,'J15': 68,'L15': 69,'R15': 70,'T15': 71,'W15': 72,'I08': 73,'I03': 74,'J08': 75,'I01': 76,'I02': 77,'J01': 78,'K01': 79,'K02': 80,'L01': 81,'L02': 82,'M01': 83,'M02': 84,'N01': 85,'N02': 86,'O01': 87,'I09': 88,'D14': 89,'D13': 90,'K07': 91,'K08': 92,'N15': 93,'P15': 94,'I15': 95,'S15': 96,'U15': 97,'V15': 98,'S07': 99,'S08': 100,'L09': 101,'L08': 102,'Q02': 103,'Q01': 104}
df_full['location'] = df_full['location'].map(dict)
df_train=df_full.sample(frac=0.8,random_state=200)
df_valid=df_full.drop(df_train.index)

location_counts = BLE_RSSI.location.value_counts()
x1 = np.asarray(df_train[FEATURES])
y1 = np.asarray(df_train['location'])

x2 = np.asarray(df_valid[FEATURES])
y2 = np.asarray(df_valid['location'])

def formatFeatures(features):
    formattedFeatures = {}
    numColumns = features.shape[1]

    for i in range(0, numColumns):
        formattedFeatures["b"+str(3001+i)] = features[:, i]

    return formattedFeatures

trainingFeatures = formatFeatures(x1)
trainingCategories = y1

testFeatures = formatFeatures(x2)
testCategories = y2

# Train Input Function
def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((trainingFeatures, y1))
    dataset = dataset.repeat(1000).batch(TF_BATCH_SIZE)
    return dataset

# Test Input Function
def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((testFeatures, y2))
    return dataset.repeat(1000).batch(TF_BATCH_SIZE)

# Provide list of GPUs should be used to train the model

distribution=tf.distribute.experimental.ParameterServerStrategy()
print('Number of devices: {}'.format(distribution.num_replicas_in_sync))

# Configuration of  training model

config = tf.estimator.RunConfig(train_distribute=distribution, model_dir=TF_MODEL_DIR, save_summary_steps=100, save_checkpoints_steps=100)

# Build 3 layer DNN classifier

model = tf.estimator.DNNClassifier(hidden_units = [13,65,110],
                 feature_columns = feature_columns,
                 model_dir = TF_MODEL_DIR,
                 n_classes=105, config=config
               )

export_final = tf.estimator.FinalExporter(TF_EXPORT_DIR, serving_input_receiver_fn=serving_input_receiver_fn)

train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, 
                                    max_steps=TF_TRAIN_STEPS)

eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn,
                                  steps=100,
                                  exporters=export_final,
                                  throttle_secs=1,
                                  start_delay_secs=1)

# Train and Evaluate the model

tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

INFO:tensorflow:ParameterServerStrategy with compute_devices = ('/device:CPU:0',), variable_device = '/device:CPU:0'


[I 200612 12:20:58 parameter_server_strategy:280] ParameterServerStrategy with compute_devices = ('/device:CPU:0',), variable_device = '/device:CPU:0'


Number of devices: 1
INFO:tensorflow:Initializing RunConfig with distribution strategies.


[I 200612 12:20:58 run_config:558] Initializing RunConfig with distribution strategies.


INFO:tensorflow:Not using Distribute Coordinator.


...

INFO:tensorflow:SavedModel written to: blerssi/export/blerssi/temp-b'1591964521'/saved_model.pb


[I 200612 12:22:01 builder_impl:421] SavedModel written to: blerssi/export/blerssi/temp-b'1591964521'/saved_model.pb


INFO:tensorflow:Loss for final step: 108.486984.


[I 200612 12:22:01 estimator:368] Loss for final step: 108.486984.


({'accuracy': 0.1962,
  'average_loss': 3.017377,
  'loss': 150.86885,
  'global_step': 5000},
 [b'blerssi/export/blerssi/1591964521'])

## Upload model folder & content into MinIO bucket

In [38]:
if model_store == 'minio':
    
    # Create a MinIO client
    minioClient = Minio(s3_endPoint,
                    access_key='minio',
                    secret_key='minio123',
                    secure=False)
    
    # Upload the model folder & contents into MinIO bucket
    initial_dir='export/' + TF_EXPORT_DIR
    for dir in os.listdir(os.path.join(TF_MODEL_DIR, initial_dir)):
        if re.match( "^[0-9]+$", dir):
            for subdir in os.listdir(os.path.join(TF_MODEL_DIR, initial_dir, dir)):
                if subdir=='variables':
                    for file in os.listdir(os.path.join(TF_MODEL_DIR, initial_dir, dir, subdir)):
                        obj_name = TF_EXPORT_DIR + dir + '/' + subdir + '/' + file
                        print(minioClient.fput_object(minio_bucket, obj_name ,os.path.join(TF_MODEL_DIR, initial_dir, dir, subdir, file)))
                else:
                    obj_name = TF_EXPORT_DIR + dir + '/' + subdir
                    print(minioClient.fput_object(minio_bucket, obj_name, os.path.join(TF_MODEL_DIR, initial_dir, dir, subdir)))
                    
    # List objects stored in minIO bucket
    model_response = minio_uploader.client.list_objects(Bucket=minio_bucket)
    print("List of objects as stored in MinIO bucket:\n\t")
    print(model_response)

9d58a142e5af9d7291cffc58aa891503
4808fd80e4e8053f392d4873726d819a
7b7837a8f93a9cd849fdc8cec89de0e1
59c6186c294e5b8a15c18f3d6e784459
List of objects as stored in MinIO bucket:
	
{'ResponseMetadata': {'RequestId': '1617CB5E388CB67C', 'HostId': '', 'HTTPStatusCode': 200, 'HTTPHeaders': {'accept-ranges': 'bytes', 'content-type': 'application/xml', 'server': 'Minio/RELEASE.2018-02-09T22-40-05Z (linux; amd64)', 'vary': 'Origin', 'x-amz-request-id': '1617CB5E388CB67C', 'date': 'Fri, 12 Jun 2020 12:23:18 GMT', 'transfer-encoding': 'chunked'}, 'RetryAttempts': 0}, 'IsTruncated': False, 'Marker': '', 'NextMarker': 'blerssi/1591964521/variables/variables.index', 'Contents': [{'Key': 'blerssi/1591964521/saved_model.pb', 'LastModified': datetime.datetime(2020, 6, 12, 12, 23, 18, 822000, tzinfo=tzlocal()), 'ETag': '"59c6186c294e5b8a15c18f3d6e784459"', 'Size': 98356, 'StorageClass': 'STANDARD', 'Owner': {'DisplayName': '', 'ID': '02d6176db174dc93cb1b899f7c6078f08654445fe8cf1b6ce98d8855f66bdbf4'}}, {'

## Define inference service name & model storage URI

In [39]:
svc_name = 'blerssi-service'

if model_store == 'minio':
    
    storageURI = "s3://" + minio_bucket + '/' + TF_EXPORT_DIR
    print(storageURI)
    
elif model_store == 'workspace-vol':
    pvcname = !(echo  $HOSTNAME | sed 's/.\{2\}$//')
    pvc = "workspace-"+pvcname[0]
    
    #storageURI = "pvc://nfs/blerssi/export/blerssi"
    if TF_MODEL_DIR.split()[0] != '/':
        MOD_TF_MODEL_DIR = '/' + TF_MODEL_DIR
    
    storageURI = "pvc://" + pvc + os.path.join(MOD_TF_MODEL_DIR, 'export/', TF_EXPORT_DIR)
    print(storageURI)

s3://minioblerssi/blerssi/


## Define configuration for inference service creation

In [40]:
if model_store == 'minio':
    
    minio_blerssi_kf = f"""apiVersion: "serving.kubeflow.org/v1alpha2"
kind: "InferenceService"
metadata:
  name: {svc_name}
  namespace: anonymous
spec:
  default:
    predictor:
      serviceAccountName: minio-sa
      tensorflow:
        storageUri: {storageURI}
"""
    
    kfserving = yaml.safe_load(minio_blerssi_kf)
    with open('blerssi-kfserving.yaml', 'w') as file:
        yaml_kfserving = yaml.dump(kfserving,file)
        
    ! cat blerssi-kfserving.yaml
    
elif model_store == 'workspace-vol':
    
    wsvol_blerssi_kf = f"""apiVersion: "serving.kubeflow.org/v1alpha2"
kind: "InferenceService"
metadata:
  name: {svc_name}
  namespace: anonymous
spec:
  default:
    predictor:
      tensorflow:
        storageUri: {storageURI}
"""
    
    kfserving = yaml.safe_load(wsvol_blerssi_kf)
    with open('blerssi-kfserving.yaml', 'w') as file:
        yaml_kfserving = yaml.dump(kfserving,file)
        
    ! cat blerssi-kfserving.yaml    

apiVersion: serving.kubeflow.org/v1alpha2
kind: InferenceService
metadata:
  name: blerssi-service
  namespace: anonymous
spec:
  default:
    predictor:
      serviceAccountName: minio-sa
      tensorflow:
        storageUri: s3://minioblerssi/blerssi/


## Apply the configuration .yaml file

By applying the configuration .yaml file, serving of BLERSSI model is done using Kubeflow KFServing.

In [41]:
!kubectl apply -f blerssi-kfserving.yaml

inferenceservice.serving.kubeflow.org/blerssi-service created


## Check whether inferenceservice is created

In [46]:
!kubectl get inferenceservice -n anonymous

NAME              URL                                                                      READY   DEFAULT TRAFFIC   CANARY TRAFFIC   AGE
blerssi-model     http://blerssi-model.anonymous.example.com/v1/models/blerssi-model       True    100                                23h
blerssi-service   http://blerssi-service.anonymous.example.com/v1/models/blerssi-service   True    100                                18s


#### Note:
Wait for inference service READY="True"

## Data preprocessing for prediction

In [47]:
df_full = pd.read_csv(os.path.join(path, 'iBeacon_RSSI_Labeled.csv')) #Labeled dataset
# Input Data Preprocessing 
df_full = df_full.drop(['date'],axis = 1)
df_full = df_full.drop(['location'],axis = 1)
df_full[FEATURES] = (df_full[FEATURES])/(-200)
print(df_full.head(5))

   b3001  b3002  b3003  b3004  b3005  b3006  b3007  b3008  b3009  b3010  \
0    1.0    1.0    1.0    1.0    1.0  0.390    1.0    1.0    1.0    1.0   
1    1.0    1.0    1.0    1.0    1.0  0.390    1.0    1.0    1.0    1.0   
2    1.0    1.0    1.0    1.0    1.0  0.385    1.0    1.0    1.0    1.0   
3    1.0    1.0    1.0    1.0    1.0  0.385    1.0    1.0    1.0    1.0   
4    1.0    1.0    1.0    1.0    1.0  0.385    1.0    1.0    1.0    1.0   

   b3011  b3012  b3013  
0    1.0    1.0    1.0  
1    1.0    1.0    1.0  
2    1.0    1.0    1.0  
3    1.0    1.0    1.0  
4    1.0    1.0    1.0  


## Predict data from serving after setting INGRESS_IP

Note - Use one of preprocessed row values from previous cell as values for "instances" in the below request.

In [48]:
!curl -v -H "Host: blerssi-service.anonymous.example.com" http://INGRESS_IP:31380/v1/models/blerssi-service:predict -d '{"signature_name":"predict","instances":[{"b3001":[-0.458086] , "b3002":[-0.6244] , "b3003":[2.354243], "b3004":[-0.404581] , "b3005":[1.421444] , "b3006":[1.767642] , "b3007":[2.637829] , "b3008":[-0.603085] , "b3009":[0.382779] , "b3010":[-0.378999] , "b3011":[-0.341798] , "b3012":[-0.303249] , "b3013":[-0.327776]}]}'

*   Trying 10.30.118.172...
* TCP_NODELAY set
* Connected to 10.30.118.172 (10.30.118.172) port 31380 (#0)
> POST /v1/models/blerssi-service:predict HTTP/1.1
> Host: blerssi-service.anonymous.example.com
> User-Agent: curl/7.58.0
> Accept: */*
> Content-Length: 320
> Content-Type: application/x-www-form-urlencoded
> 
* upload completely sent off: 320 out of 320 bytes
< HTTP/1.1 200 OK
< content-length: 4223
< content-type: application/json
< date: Fri, 12 Jun 2020 12:24:36 GMT
< x-envoy-upstream-service-time: 166
< server: istio-envoy
< 
{
    "predictions": [
        {
            "class_ids": [9],
            "classes": ["9"],
            "all_class_ids": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,

# Cleanup after prediction

## Delete KFserving model

In [49]:
!kubectl delete -f blerssi-kfserving.yaml

inferenceservice.serving.kubeflow.org "blerssi-service" deleted


In [50]:
if model_store == 'minio':
    
    #Delete minIo secret
    !kubectl delete -f minio-secret.yaml
    
    #Define minIO service account
    !kubectl delete -f minio-serviceaccount.yaml
    
    #Delete minIO objects & minIO bucket
    model_response = minio_uploader.client.list_objects(Bucket=minio_bucket)
    
    #Delete locally stored model folder
    !rm -rf $TF_MODEL_DIR

    obj_list = []
    for obj_name in model_response['Contents']:
        obj_list.append({'Key' : obj_name['Key']})
        
    print("Deleting the stored objects in minIO bucket.....")
    minio_uploader.client.delete_objects(Bucket=minio_bucket, Delete={'Objects' : obj_list})
    
    print("Deleting the minIO bucket.....")
    minio_uploader.client.delete_bucket(Bucket=minio_bucket)
    print("Done")
    
elif model_store == 'workspace-vol':
    
    #Delete the model folder
    print("Deleting model folder...")
    !rm -rf $TF_MODEL_DIR
    print("Done")

secret "miniosecret" deleted
serviceaccount "minio-sa" deleted
Deleting the stored objects in minIO bucket.....
Deleting the minIO bucket.....
Done
